In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [ ]:
spark = SparkSession.builder.master('local').appName('operDataFrames').enableHiveSupport().getOrCreate()
sc=spark.sparkContext

In [ ]:
orders = spark.read.csv('retail_db//orders.csv',header=True,inferSchema=True)
order_items = spark.read.csv('retail_db//order_items.csv',header=True,inferSchema=True)
customers = spark.read.csv('retail_db//customers.csv',header=True,inferSchema=True)
products = spark.read.csv('retail_db//products.csv',header=True,inferSchema=True)

# Analytical functions 

### It is driven by windowing function 'Window' imported from 'pyspark.sql.window'

### Basic analytical functions 

These analytical functions are count(), sum(), min(), max() and avg()


Only partition by clause is applicable (No order by clause is needed)

#### Define the specification for windowing according to the problem statement

In [ ]:
spec=Window.partitionBy(order_items.order_item_order_id)

In [ ]:
order_items.withColumn('order_sum',sum(order_items.order_item_subtotal).over(spec)).show()

In [ ]:
order_items. \
withColumn('order_sum',sum(order_items.order_item_subtotal).over(spec)). \
withColumn('order_max',max(order_items.order_item_subtotal).over(spec)). \
withColumn('order_min',min(order_items.order_item_subtotal).over(spec)). \
withColumn('order_avg',avg(order_items.order_item_subtotal).over(spec)). \
withColumn('order_count',count(order_items.order_item_subtotal).over(spec)). \
withColumn('order_count1',count(lit("1")).over(spec)). \
drop('order_item_product_id','order_item_quantity','order_item_product_price'). \
orderBy('order_item_order_id').show()

## Numbering Analytical functions 

These analytical functions are rank(), dense_rank(), row_number(), lead(), lag() and ntile()

Both partitionBy and orderBy clauses must be specified for windowing 

In [ ]:
spec=Window.partitionBy(order_items.order_item_order_id).orderBy(order_items.order_item_subtotal.desc())

In [ ]:
order_items.withColumn('rnk',rank().over(spec)).show()

In [ ]:
order_items. \
withColumn('rnk',rank().over(spec)). \
withColumn('drnk',dense_rank().over(spec)). \
withColumn('rn',row_number().over(spec)). \
withColumn('ld',lead(order_items.order_item_subtotal).over(spec)). \
withColumn('lg',lag(order_items.order_item_subtotal).over(spec)). \
drop('order_item_product_id','order_item_quantity','order_item_product_price'). \
orderBy('order_item_order_id').show()